### Below code is used so that system points to my virtual environment, kindly adjust it according to your system 

In [734]:
import sys
sys.path.append("/Users/yatingupta/Github/Reddit-flare-detector/env/lib/python3.7/site-packages")

In [735]:
#https://www.labnol.org/internet/web-scraping-reddit/28369/ [Can be used if more tha 1000 posts are required]

In [736]:
#Necessary libraries
import praw
import pandas as pd
import matplotlib
import string
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer    
from nltk.corpus import stopwords
set(stopwords.words('english'))
ps = PorterStemmer() 

In [737]:
#Downloading lib
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/yatingupta/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/yatingupta/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [738]:
#Id to scrape
my_client_id = "tlaYd7tsDOqvlQ"
my_client_secret = "xdRqwLkA07r8ScyJZTMsYUndrSA"
my_user_agent = "scrapping r/india"

In [739]:
#Helper function to print full panda data frame to analyze data
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    pd.set_option('display.max_colwidth', 950)
    print(x)
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_colwidth')

In [740]:
# API to access reddit data
reddit = praw.Reddit(client_id=my_client_id, client_secret=my_client_secret, user_agent=my_user_agent)
india_subreddit = reddit.subreddit('India')

## Data Collection
<ul>
<li>Getting TOP, HOT and NEW posts of reddit using the praw API </li>
<li>Many JSON tags are retrived which may or may not be used in future </li>
<li>https://github.com/reddit-archive/reddit/wiki/JSON used to study the tags </li>
</ul>

In [741]:
top_posts = []
for post in india_subreddit.top(limit=1000):
    top_posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created,post.link_flair_text])

hot_posts = []
for post in india_subreddit.hot(limit=1000):
    hot_posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created,post.link_flair_text])


new_posts = []
for post in india_subreddit.new(limit=1000):
    new_posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created,post.link_flair_text])


## Exploratory Data Analysis

In [742]:
top_posts = pd.DataFrame(top_posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created','flare'])
top_text_flare_data = top_posts[['id','title','body','flare']]

hot_posts = pd.DataFrame(hot_posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created','flare'])
hot_text_flare_data = hot_posts[['id','title','body','flare']]

new_posts = pd.DataFrame(new_posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created','flare'])
new_text_flare_data = new_posts[['id','title','body','flare']]


In [743]:
top_text_flare_data = top_posts[['id','title','body','flare']]
hot_text_flare_data = hot_posts[['id','title','body','flare']]
new_text_flare_data = new_posts[['id','title','body','flare']]

frames = [top_text_flare_data, hot_text_flare_data, new_text_flare_data]
data = pd.concat(frames)
data.drop_duplicates(keep='last',inplace=True)
data.describe()
data['flare'].value_counts()

Non-Political                       531
Coronavirus                         417
Politics                            400
AskIndia                            156
Photography                          51
Policy/Economy                       46
[R]eddiquette                        35
Sports                               35
Business/Finance                     34
Science/Technology                   31
Unverified                           16
Food                                 16
CAA-NRC                              10
Scheduled                             9
r/all                                 3
CAA-NRC-NPR                           3
AMA                                   2
Demonetization                        2
/r/all                                2
Policy & Economy                      2
On Internet Shutdowns                 1
Misleading Headline                   1
Original Comics                       1
Misleading                            1
40 Martyrs                            1


In [744]:
top_text_flare_data.describe()

,id,title,body,flare
count,988,988,988,987
unique,988,988,65,35
top,debi1z,Spotted In Bombay Protests Today,,Non-Political
freq,1,1,924,422


In [745]:
hot_text_flare_data.describe()

,id,title,body,flare
count,780,780,780,776
unique,780,771,237,15
top,g1nd9a,Late Night Random Discussion Thread !,,Coronavirus
freq,1,3,539,349


In [746]:
new_text_flare_data.describe()

,id,title,body,flare
count,837,837,837,833
unique,837,827,257,16
top,g1q22s,Late Night Random Discussion Thread !,,Coronavirus
freq,1,4,575,371


## Building the Flare Detector

<ul>
    <li> The required data here is extracted from data collected earlier </li>
    <li> The TOP, HOT and NEW posts are concatenated in frame </li>
    <li> duplicates are removed </li>
</ul>    

In [747]:
top_text_flare_data = top_posts[['id','title','body','flare']]
hot_text_flare_data = hot_posts[['id','title','body','flare']]
new_text_flare_data = new_posts[['id','title','body','flare']]

frames = [top_text_flare_data, hot_text_flare_data, new_text_flare_data]
data = pd.concat(frames)
data.drop_duplicates(keep='last',inplace=True)
data.describe()

,id,title,body,flare
count,1825,1825,1825,1820
unique,1825,1815,324,39
top,a9qr7h,Late Night Random Discussion Thread !,,Non-Political
freq,1,4,1496,531


### Since the number of flare was large (40), all the flares with less than 3 posts are deleted here.

In [748]:
data = data.groupby('flare').filter(lambda x : len(x)>3)
data.describe()

,id,title,body,flare
count,1787,1787,1787,1787
unique,1787,1777,316,14
top,a9qr7h,Late Night Random Discussion Thread !,,Non-Political
freq,1,4,1466,531


<ul>
<li>Creating a text column which concatenates the title and body text </li>
<li> ALL the rows with NONE data are removed </li>
</ul>

In [749]:
data['text'] = data['title'].str.cat(data['body'], sep =" ")
data = data.mask(data.eq('None')).dropna()
data.describe()

,id,title,body,flare,text
count,1787,1787,1787,1787,1787
unique,1787,1777,316,14,1779
top,a9qr7h,Late Night Random Discussion Thread !,,Non-Political,Late Night Random Discussion Thread ! ^Beep ^B...
freq,1,4,1466,531,3


#### Preprocessing the data
<ul>
    <li>Creating regex emoji pattern to remove emojis</li>
    <li> All strings are lowered </li>
    <li> Punctuation, whitespaces,links are removed</li>
    <li> Stop words are removed and words are stemmed</li>
    <li> Finally it contains list of strings where each word is stemmed and cleaned</li>
</ul>

In [750]:
emoji_pattern = re.compile("["
                       u"\U0001F600-\U0001F64F"  # emoticons
                       u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                       u"\U0001F680-\U0001F6FF"  # transport & map symbols
                       u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                       u"\U00002702-\U000027B0"
                       u"\U000024C2-\U0001F251"
                       "]+", flags=re.UNICODE)

data['text'] = (data['text'].str.lower() #lowercase
                           .str.replace(r'[^\w\s]+', '') #rem punctuation 
                           .str.replace(emoji_pattern, '') #rem emoji
                           .str.replace(r'http\S+','') #rem links
                           .str.strip() #rem trailing whitespaces
                           .str.split()) #split by whitespaces

res = []
stop_words = set(stopwords.words('english')) 

for i in data['text']:
    t = ""
    for j in i:
        if j not in stop_words:
            w = ps.stem(j)
            t += w
            t += " "
    res.append(t)
data['text'] = res

Split the dataset into training and validation datasets 


In [752]:
from sklearn import model_selection
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer

train_x, test_x, train_y, test_y = model_selection.train_test_split(data['text'], data['flare'],test_size=0.3)

# X = the entire text data , Y = entire target or flares
X = data['text']
Y = data['flare']

encoder = preprocessing.LabelEncoder()

# label encode the target variable 
Y = encoder.fit_transform(Y)
train_y = encoder.transform(train_y)
test_y = encoder.transform(test_y)


Converting the words to vectors using Tfidf to be used for ML algos

In [753]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(data['text'])
Train_X_Tfidf = Tfidf_vect.transform(train_x)
Test_X_Tfidf = Tfidf_vect.transform(test_x)
X = Tfidf_vect.transform(X)

### Calculating Accuracy of different ML algo

#### Naive Bayes

In [754]:
from sklearn import naive_bayes
# fit the training dataset on the NB classifier
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,train_y)
# predict the labels on validation dataset
predictions_NB = Naive.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
accuracy_naive = accuracy_score(predictions_NB, test_y)*100

print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, test_y)*100)

Naive Bayes Accuracy Score ->  54.00372439478585


#### SVM

In [755]:
from sklearn import svm
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,train_y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
accuracy_SVM = accuracy_score(predictions_SVM, test_y)*100
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, test_y)*100)

SVM Accuracy Score ->  56.61080074487895


#### Random Forest Classifier

In [756]:
from sklearn import ensemble
random_forest_classfier = ensemble.RandomForestClassifier()
random_forest_classfier.fit(Train_X_Tfidf,train_y)
predictions_RF = random_forest_classfier.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
accuracy_rfc = accuracy_score(predictions_RF, test_y)*100
print("RF Accuracy Score -> ",accuracy_score(predictions_RF, test_y)*100)

RF Accuracy Score ->  53.81750465549349


#### SGDClassifier 
linear classifier optimized by the SGD.

In [757]:
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)
sgd.fit(Train_X_Tfidf,train_y)
# predict the labels on validation dataset
predictions_sgd = sgd.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
accuracy_sgd = accuracy_score(predictions_sgd, test_y)*100
print("SGD Accuracy Score -> ",accuracy_score(predictions_sgd, test_y)*100)

SGD Accuracy Score ->  57.16945996275605


#### LogisticRegression

In [758]:
from sklearn.linear_model import LogisticRegression
Logistic_Regression = LogisticRegression(n_jobs=1, C=1e5,max_iter=2000)
Logistic_Regression.fit(Train_X_Tfidf,train_y)
# predict the labels on validation dataset
predictions_Logistic_Regression = Logistic_Regression.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
accuracy_lr = accuracy_score(predictions_Logistic_Regression, test_y)*100
print("Logistic_Regression Accuracy Score -> ",accuracy_score(predictions_Logistic_Regression, test_y)*100)

Logistic_Regression Accuracy Score ->  53.63128491620112


### Accuracy Comparison

In [759]:
model_prediction = [accuracy_naive,accuracy_SVM,accuracy_rfc,accuracy_sgd,accuracy_lr]
model_names = ["Naive","SVM","random_forest_classfier","sgd","Logistic_Regression"]
for i in range(0,len(models)):
    print("Accuracy by " + model_names[i] + " = " + str(model_prediction[i]) + "%")

Accuracy by Naive = 54.00372439478585%
Accuracy by SVM = 56.61080074487895%
Accuracy by random_forest_classfier = 53.81750465549349%
Accuracy by sgd = 57.16945996275605%
Accuracy by Logistic_Regression = 53.63128491620112%


In [760]:
""""
kfold = model_selection.KFold(n_splits=10)
model = ensemble.RandomForestClassifier()
results = model_selection.cross_val_score(model, X, Y, cv=kfold)
print("Accuracy: %.3f%% (%.3f%%)" % (results.mean()*100.0, results.std()*100.0))

"""""

'"\nkfold = model_selection.KFold(n_splits=10)\nmodel = ensemble.RandomForestClassifier()\nresults = model_selection.cross_val_score(model, X, Y, cv=kfold)\nprint("Accuracy: %.3f%% (%.3f%%)" % (results.mean()*100.0, results.std()*100.0))\n\n'

#### Testing on Manual Examples
below example list is created on which the models will predict the flare

In [761]:
example_list = np.array(["Stuck in quarantine. Playing games all day","This. virus is killing me","Government of India needs to take action ","All the political ministers need to work for india"])
Example = pd.Series(example_list)
Example = (Example.str.lower() #lowercase
                           .str.replace(r'[^\w\s]+', '') #rem punctuation 
                           .str.replace(emoji_pattern, '') #rem emoji
                           .str.replace(r'http\S+','') #rem links
                           .str.strip() #rem trailing whitespaces
                           .str.split()) #split by whitespaces
res = []
stop_words = set(stopwords.words('english')) 

for i in Example:
    t = ""
    for j in i:
        if j not in stop_words:
            w = ps.stem(j)
            t += w
            t += " "
    res.append(t)
Example = res
Example_Tfidf = Tfidf_vect.transform(Example)

In [762]:
models = [Naive,SVM,random_forest_classfier,sgd,Logistic_Regression]
for i in range(0,len(models)):
    model = models[i]
    predict_example = model.predict(Example_Tfidf)
    print("Predictions by " + model_names[i])
    print(encoder.inverse_transform(predict_example))


Predictions by Naive
['Non-Political' 'Coronavirus' 'Coronavirus' 'Coronavirus']
Predictions by SVM
['Non-Political' 'Coronavirus' 'Coronavirus' 'Politics']
Predictions by random_forest_classfier
['Non-Political' 'Politics' 'Politics' 'Politics']
Predictions by sgd
['Non-Political' 'Non-Political' 'Politics' 'Politics']
Predictions by Logistic_Regression
['Non-Political' 'Policy/Economy' 'Coronavirus' 'Politics']


#### Below code will download the model, tfidf vectorizer and encoder to be directly used in Web App

In [763]:
import pickle
pickle.dump(model, open("model.pkl","wb"))
pickle.dump(Tfidf_vect, open("tfidf.pickle", "wb"))
np.save('classes.npy', encoder.classes_)

#### This is used to test the endpoint by sending request from test.txt

In [764]:
import requests

with open('test.txt', 'rb') as f:
    r = requests.post('http://127.0.0.1:5000/automated_testing', files={'test.txt': f})
    a = r.json()
    print(a)

{'https://www.reddit.com/r/india/comments/fuug0q/ive_never_felt_so_helpless_please_help_our_family/?utm_medium=android_app&utm_source=share': 'Coronavirus', 'https://www.reddit.com/r/india/comments/fzobyi/anyone_else_sick_of_news_reporting_what/?utm_medium=android_app&utm_source=share': 'Non-Political', 'https://www.reddit.com/r/nextfuckinglevel/comments/g289tw/the_needle_galaxy_is_nearly_50_million_lightyears/': 'Non-Political'}
